# SWMAL Exercise

## Advanced CNN using Roboflow

In this exercise we explore object detection machine learning algorithms, using Roboflow and YOLOv8 API. YOLOv8 is an api developed by Ultralytics, which is designed for CNN models. It is a successor of YOLOv3 and YOLOv5, and has proven itself to be a powerful framework for tasks such as object detection and classification. Because of this we will use it for the most glorious purpose imaginable: detecting cats! And if you hate cats, we can use it to crop them out of the pictures...

## Creating a dataset to work with

In order to detect cats properly, one must first create a dataset of images which the model can work with. We will start with cloning an appropriate project from https://universe.roboflow.com/ . For this project we chose a project which already has 50 images to work with, and luckily for us they are annotated. Cloning the images is quite straightforward - you select the project to clone from, choose the images to clone and your Roboflow workspace/project where the images will be placed.

![cloning in roboflow](cnn2_Imgs/roboflow_cloning.png)





In this case, the images which we have cloned are already annotated: there are bounding boxes around the cats. This is helpful, because it will spare us the time to go through each of these images individually and annotate them ourselves. However, to illustrate how such process is done we decided to upload a few images of our own, and annotate them.

![roboflow annotations](cnn2_imgs/roboflow_annotation.png)




We then generate a new version of the dataset. This step performs the Train/Test split and some preprocessing on the images, notably by downsizing them since ML models tend to work better on smaller pixel resolution images.

![roboflow version generation](cnn2_imgs/roboflow_version.png)

This also creates a code snippet that downloads the generated dataset.


## Yolo CLI and SDK intro

To make sure that everything is setup correctly we will go through the tutorial on https://blog.roboflow.com/how-to-train-yolov8-on-a-custom-dataset/ . This also introduces us briefly to YOLO SDK in python.

In [1]:
# Checks that we have access to the GPU
! nvidia-smi

from IPython import display
display.clear_output()

# Check that yolo is installed properly by running this CLI command. If no exception is thrown, we are golden.
! yolo mode="track"
# YOLO installs some missing dependencies on its own when running via CLI! Neat!






WARNING ⚠️ 'model' is missing. Using default 'model=yolov8n.pt'.
WARNING ⚠️ 'source' is missing. Using default 'source=/home/swmal10e23/.local/lib/python3.9/site-packages/ultralytics/assets'.
Ultralytics YOLOv8.0.209 🚀 Python-3.9.12 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 3060, 12043MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

image 1/2 /home/swmal10e23/.local/lib/python3.9/site-packages/ultralytics/assets/bus.jpg: 640x480 3 persons, 1 bus, 7.5ms
image 2/2 /home/swmal10e23/.local/lib/python3.9/site-packages/ultralytics/assets/zidane.jpg: 384x640 2 persons, 8.1ms
Speed: 1.0ms preprocess, 7.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/track


In [2]:
# test SDK
from ultralytics import YOLO
from PIL import Image

# Download and show the dog picture
! curl https://media.roboflow.com/notebooks/examples/dog.jpeg -o ./dog.jpeg
img = Image.open("./dog.jpeg")
img.show()

# Test on the tutorial example with a dog. pt = model is pretrained
model = YOLO('yolov8n.pt')
results = model.predict(source="https://media.roboflow.com/notebooks/examples/dog.jpeg", conf=0.25)




  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  103k  100  103k    0     0  1135k      0 --:--:-- --:--:-- --:--:-- 1138k


Error: no "view" rule for type "image/png" passed its test case
       (for more information, add "--debug=1" on the command line)


>4;1H8 D)ownload, or C)ancel                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          


Found https://media.roboflow.com/notebooks/examples/dog.jpeg locally at dog.jpeg
image 1/1 /home/swmal10e23/swmal_grp10/O3/dog.jpeg: 640x384 1 person, 1 car, 1 dog, 8.0ms
Speed: 0.9ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)


The algorithm found even the blurred car in the picture, which I myself have missed. Quite impressive.

It's time to train and test a YOLO model on our cat dataset. The first step is to load the data from Roboflow.

## Loading and training on data from Roboflow

We will use the code generated earlier by Roboflow to download the data from it. It uses roboflows own package and api to download data securely.

In [3]:
# If you haven't installed roboflow package:
# ! pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="RcblOwaZETwq19a8fyov")
project = rf.workspace("swmal10").project("catfinder-hh2e7")
dataset = project.version(1).download("yolov8")

# You must change the paths in project/data.yaml : all paths are relative to data.yaml directory, but roboflow mistakingly generates paths with project dir... and test path goes back for no reason


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.209, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to CatFinder-1 in yolov8:: 100%|██████████| 122/122 [00:00<00:00, 167.94it/s]


In [ ]:
from ultralytics import YOLO
import os
import platform
newmodel = YOLO("yolov8n.pt")

# Find the directory to the data.yaml file. We need to change its contents because Roboflow
# puts wrong paths in data.yaml. This might be due to incorrect versions? Roboflow
# complains about yolo lib being too new...



def fix_my_yaml(path):
    print(path)
    # Replace the incorrect paths in .yaml
    lines = []
    with open (path) as f:
        for line in f:
            lines.append(line)
            # print(line)

    with open (datapath, "w") as f:
        for s in lines:
            if "../test" in s:
                s = s.replace("../", "")
                f.write(s)
            elif "train:" in s:
                s = s.replace("CatFinder-1/", "")
                f.write(s)
            elif "val:" in s:
                s = s.replace("CatFinder-1/", "")
                f.write(s)
            else:
                f.write(s)
    print("Fixed data.yaml in ", datapath)

# Don't run on Windows
if platform.system() == "Linux":
    ! find / -iname "CatFinder-1"  2>&1 | grep -v "Permission denied" | grep -v "Invalid argument" > path.txt
    with open("path.txt") as f: s = f.readline()[:-1] # -1 because find returns \n at the end
    os.remove("path.txt")
    datapath = s + "/data.yaml"
    fix_my_yaml(datapath)
    res = newmodel.train(data=datapath)

elif platform.system() == "Windows":
    # Please insert your own path and make sure the .yaml file is in order
    res = newmodel.train(data="C:\\UNI_2023\\ml\\swmal_grp10\\O3\\CatFinder-1\\data.yaml")

# The more epochs, the better: I run 100 on my desktop but 10 on my laptop
# If on gpu cluster/some other server you can increase epochs


# This crashed on my computer every time for some reason...
# The reason was OBVIOUSLY one of the dependencies. Probably matplotlib or scipy (called by seaborn, but seaborn was not the problem)
# Solved by upgrading via python -m pip install <package> --upgrade
print(res)


All the results are stored in runs/train/ directory and include labeled images as well as a lot of statistical data about the models performance.


In [ ]:


from roboflow import Roboflow
rf = Roboflow(api_key="RcblOwaZETwq19a8fyov")
project = rf.workspace("swmal10").project("catfinder-hh2e7")
dataset = project.version(2).download("yolov8")

This downloads the dataset into CatFinder-num directory. Note that default roboflow installation has data.yaml file with incorrect paths and need to be changed.  
test: ../test/images -> test: test/images  
train: CatFinder-1/train/images -> train: train/images  
val: CatFinder-1/valid/images -> val: valid/images  

In [2]:
from ultralytics import YOLO
newmodel = YOLO('yolov8n.pt')
# Train the model
newmodel.train(data="/home/swmal10e23/swmal_grp10/O3/CatFinder-2/data.yaml")

# Windows paths
# res = newmodel.train(data="C:\\UNI_2023\\ml\\swmal_grp10\\O3\\CatFinder-2\\data.yaml")
# res = newmodel.predict(source="C:\\UNI_2023\\ml\\swmal_grp10\\O3\\CatFinder-2\\test\\images")


Ultralytics YOLOv8.0.209 🚀 Python-3.9.12 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 3060, 12043MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/swmal10e23/swmal_grp10/O3/CatFinder-2/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, ag

  0%|          | 0.00/755k [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

'GrouperView' object has no attribute 'join'


Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train6
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      1/100      2.28G      1.122      2.633      1.455         40        640: 100%|██████████| 20/20 [00:02<00:00,  7.80it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.05it/s]
                   all         36         55      0.782        0.2      0.488      0.259

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      2/100      2.36G       1.12      1.761      1.483         49        640: 100%|██████████| 20/20 [00:02<00:00,  9.66it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 14.25it/s]
                   all         36         55      0.403      0.327      0.398      0.201

     

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f7d23b88d00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

The results of the trained model can be found in './runs/train/' directory. Let's try to predict where the cat is on a picture from outside the dataset!

In [11]:

newmodel.predict(source="/home/swmal10e23/swmal_grp10/O3/kitty-catty.jpeg", save=True)



image 1/1 /home/swmal10e23/swmal_grp10/O3/kitty-catty.jpeg: 640x640 2 CATs, 4.9ms
Speed: 16.4ms preprocess, 4.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/train63


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'CAT', 1: 'cats'}
 orig_img: array([[[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  6],
         [ 0,  0,  6],
         [ 0,  0,  6]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  6],
         [ 0,  0,  6],
         [ 0,  0,  6]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  6],
         [ 0,  0,  6],
         [ 0,  0,  6]],
 
        ...,
 
        [[35, 51, 74],
         [34, 50, 73],
         [31, 47, 70],
         ...,
         [ 3,  6, 11],
         [ 3,  4,  8],
         [ 2,  3,  7]],
 
        [[36, 52, 75],
         [34, 50, 73],
         [31, 47, 70],
         ...,
         [ 5,  8, 13],
         [ 5,  6, 10],
         [ 3,  4,  8]],
 
        [[34, 50, 73],
       

The result:
![katty](runs/detect/train11/kitty-catty.jpg)

That's not bad! Although we only have a single cat on this picture, so it's not perfect either.